MUFG Loan Terms:
Loan Amount : 1,425,000
Tenor : 4 + 15
Interest: 1.15%
Upfront Fee: 0.50%
ECA Fee: 0.10%
Afent Fee: 75,000 p.a

**1. Loan Terms**

In [ ]:
!pip install numpy-financial

In [7]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy_financial as npf

# Basic Loan Terms
loan_amount = 1425000000  # Loan amount in EUR
tenor_grace = 4  # Grace period in years (no capital repayment)
tenor_total = 15  # Repayment period in years
interest_rate = 0.015  # Annual interest rate (1.5%)
repayments_per_year = 2  # Semi-annual payments
days_in_year = 365  # Using 365 days for interest calculations
insurance_fee_percent = 0.10  # Insurance rate (10%)

# Fees
eca_fee_percent = 0.001  # 0.10%
arranger_fee_percent = 0.006  # 0.60%
upfront_fee_percent = 0.005  # 0.50%

# Loan Signing Date (T0)
signing_date = datetime(2025, 1, 1)  # January 1st, 2025

# Insurance calculation
insurance_loan = (loan_amount / (1 - insurance_fee_percent)) - loan_amount  # Correct formula
total_loan_with_insurance = loan_amount + insurance_loan  # New total loan including insurance

# Calculate the fees based on the total loan amount with insurance
eca_fee = total_loan_with_insurance * eca_fee_percent
arranger_fee = total_loan_with_insurance * arranger_fee_percent
upfront_fee = total_loan_with_insurance * upfront_fee_percent
total_fees = eca_fee + arranger_fee + upfront_fee

# Calculating the number of total periods
grace_periods = tenor_grace * repayments_per_year  # 8 periods for grace
repayment_periods = tenor_total * repayments_per_year  # 30 periods for repayment
total_periods = grace_periods + repayment_periods  # Total of 38 periods

# Function to calculate the interest payment on a 365-day basis using actual days between periods
def calculate_interest_payment_365(outstanding_balance, interest_rate, days_between):
    return outstanding_balance * interest_rate * (days_between / days_in_year)

# Equal Capital Payment during the repayment period
equal_capital_payment = total_loan_with_insurance / repayment_periods

# ---------------------------------------------------
# Cash Flow Calculation
# ---------------------------------------------------

# Define the headings for the cash flow DataFrame in the desired order
headings = ['Period', 'Date', 'Disbursement', 'Upfront Fee', 'ECA Fee', 'Arranger Fee', 'Interest Payment', 'Capital Payment', 'Total Payment', 'Outstanding Balance', 'CashFlow']
cashflow_df = pd.DataFrame(columns=headings)

# Initial balance and interest calculation
outstanding_balance = insurance_loan  # Start with insurance loan included
disbursement_per_period = loan_amount / grace_periods  # Equal disbursement over grace period

previous_date = signing_date  # To calculate days between payment dates

# Add the T0 period with no disbursement or interest payment, but including upfront fees as cash outflow
cashflow_df.loc[len(cashflow_df)] = [0, signing_date.strftime('%Y-%m-%d'), 0, upfront_fee, eca_fee, arranger_fee, 0, 0, total_fees, outstanding_balance, -total_fees]

# Grace period: Disbursements start after T0, interest payments calculated from T2 onwards
for period in range(1, grace_periods + 1):
    payment_date = signing_date + relativedelta(months=+6 * period)  # Disbursements start at T1
    disbursement = disbursement_per_period  # Equal disbursement during grace period
    outstanding_balance += disbursement  # Accumulate loan balance

    # Calculate days between the previous payment and the current payment
    days_between = (payment_date - previous_date).days
    previous_date = payment_date  # Update previous date to current date

    # Interest payment starts from T2, based on the outstanding balance of T1
    interest_payment = 0 if period == 1 else calculate_interest_payment_365(outstanding_balance - disbursement, interest_rate, days_between)

    # During grace period, no capital payment
    capital_payment = 0
    total_payment = interest_payment + capital_payment
    cash_flow = disbursement - total_payment  # Disbursement is positive, payments are negative

    # Add the period's data to the DataFrame
    cashflow_df.loc[len(cashflow_df)] = [period, payment_date.strftime('%Y-%m-%d'), disbursement, 0, 0, 0, interest_payment, capital_payment, total_payment, outstanding_balance, cash_flow]

# After grace period: Repayment period starts (equal capital + interest payments)
for period in range(grace_periods + 1, total_periods + 1):
    payment_date = signing_date + relativedelta(months=+6 * period)  # Payments every 6 months

    # Calculate days between the previous payment and the current payment
    days_between = (payment_date - previous_date).days
    previous_date = payment_date  # Update previous date to current date

    # Calculate interest payment on the outstanding balance from the previous period
    interest_payment = calculate_interest_payment_365(outstanding_balance, interest_rate, days_between)

    # Capital payment remains constant
    capital_payment = equal_capital_payment
    total_payment = interest_payment + capital_payment

    # Update the outstanding balance
    outstanding_balance -= capital_payment
    cash_flow = -total_payment  # Repayments are negative cash flows

    # Add the period's data to the DataFrame
    cashflow_df.loc[len(cashflow_df)] = [period, payment_date.strftime('%Y-%m-%d'), 0, 0, 0, 0, interest_payment, capital_payment, total_payment, outstanding_balance, cash_flow]

# Add the Total Row
cashflow_df.loc[len(cashflow_df)] = ['Total', '', cashflow_df['Disbursement'].sum(), cashflow_df['Upfront Fee'].sum(), cashflow_df['ECA Fee'].sum(), cashflow_df['Arranger Fee'].sum(), cashflow_df['Interest Payment'].sum(), cashflow_df['Capital Payment'].sum(), cashflow_df['Total Payment'].sum(), '', cashflow_df['CashFlow'].sum()]

# Calculate IRR based on CashFlow
cashflow_list = cashflow_df['CashFlow'].iloc[:-1].tolist()  # Exclude the total row
irr = npf.irr(cashflow_list)

# Display the IRR
print(f"Internal Rate of Return (IRR): {2*irr*100:.2f}%")

# Display the DataFrame with cash flow and total row
cashflow_df

Internal Rate of Return (IRR): 2.94%


,Period,Date,Disbursement,Upfront Fee,ECA Fee,Arranger Fee,Interest Payment,Capital Payment,Total Payment,Outstanding Balance,CashFlow
0,0,2025-01-01,0.000000e+00,7.916667e+06,1.583333e+06,9500000.0,0.000000e+00,0.000000e+00,1.900000e+07,158333333.333333,-1.900000e+07
1,1,2025-07-01,1.781250e+08,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,336458333.333333,1.781250e+08
2,2,2026-01-01,1.781250e+08,0.000000e+00,0.000000e+00,0.0,2.544178e+06,0.000000e+00,2.544178e+06,514583333.333333,1.755808e+08
3,3,2026-07-01,1.781250e+08,0.000000e+00,0.000000e+00,0.0,3.827654e+06,0.000000e+00,3.827654e+06,692708333.333333,1.742973e+08
4,4,2027-01-01,1.781250e+08,0.000000e+00,0.000000e+00,0.0,5.238014e+06,0.000000e+00,5.238014e+06,870833333.333333,1.728870e+08
5,5,2027-07-01,1.781250e+08,0.000000e+00,0.000000e+00,0.0,6.477568e+06,0.000000e+00,6.477568e+06,1048958333.333333,1.716474e+08
6,6,2028-01-01,1.781250e+08,0.000000e+00,0.000000e+00,0.0,7.931849e+06,0.000000e+00,7.931849e+06,1227083333.333333,1.701932e+08
7,7,2028-07-01,1.781250e+08,0.000000e+00,0.000000e+00,0.0,9.177911e+06,0.000000e+00,9.177911e+06,1405208333.333333,1.689471e+08
8,8,2029-01-01,1.781250e+08,0.000000e+00,0.000000e+00,0.0,1.062568e+07,0.000000e+00,1.062568e+07,1583333333.333333,1.674993e+08
9,9,2029-07-01,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1.177740e+07,5.277778e+07,6.455518e+07,1530555555.555556,-6.455518e+07
